In [1]:
# %load_ext autoreload
# %autoreload 2
import pandas as pd
import matplotlib.colors as colors
import lib
import bioframe as bf
import dataclasses
import dataclasses
import functools
import itertools
import pathlib
from typing import Iterable, Union
import uuid
import bioframe as bf
import higlass as hg
import ipywidgets
import jscatter
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import traitlets
from IPython.display import display

In [2]:
def _encode(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj


@dataclasses.dataclass
class DynamicTileset:
    name: str
    chromsizes: pd.Series
    uid: str = dataclasses.field(default_factory=lambda: uuid.uuid4().hex)
    datatype = "bedlike"
        
    def __post_init__(self):
        abslen = self.chromsizes.cumsum()
        starts = pd.Series([0] + abslen[:-1].tolist(), index=abslen.index)
        self._starts =  dict(starts)
        self._tiles = []
        
    def update(self, df):
        starts = self._starts
        df = bf.cluster(df).groupby("cluster").agg({
            "chrom": "first",
            "start": "min",
            "end": "max",
            "name": "first",
            "score": "first",
            "strand": "first",
            "thickStart": "min",
            "thickEnd": "max",
            "rgb": "first",
        })
        self._tiles = [{
            "chrOffset": int(starts[r[0]]),
            "xStart": int(starts[r[0]] + r[1]),
            "xEnd": int(starts[r[0]] + r[2]),
            "importance": 0,
            "uid": uuid.uuid4().hex,
            "fields": tuple(map(_encode, r))
        } for r in df.to_records(index=False)]
        
    def info(self):
        genome_length = int(np.sum(self.chromsizes.values))
        return {
            "uuid": self.uid,
            "max_width": genome_length,
            "min_pos": [1],
            "max_pos": [genome_length],
            "max_zoom": 0,
        }
     
    def tiles(self, _tileids):
        return [(f"{self.uid}.0.0", self._tiles)]


In [3]:
df = pd.read_parquet("/Users/thomas/joint-pca-hic/data/hepatic_differentiation_2024_02_07_09_07_PCA-32_50000bp_hg38_post_processed_with_ipg.pq")
samples = list(df.name.unique())
df["sample"] = df.name
df["start"] = df["start"].astype(int)
df["end"] = df["end"].astype(int)
df.head()

,chrom,start,end,weight,good_bin,filename,PCA1,PCA2,PCA3,PCA4,...,umap2_n500,name,total_distance,arm,armlen,centel,centel_abs,IPG,IPGint,sample
18,chr1,900000,950000,0.007896,True,DE-FA-DSG-DdeI-DpnII-P1P2.hg38.mapq_30.1000.mcool,0.007601,0.003599,0.003639,0.000868,...,10.306364,DE,0.009066,chr1p,123400000,0.992301,122450000,A2,1,DE
19,chr1,950000,1000000,0.008673,True,DE-FA-DSG-DdeI-DpnII-P1P2.hg38.mapq_30.1000.mcool,0.007159,0.003471,0.003692,0.000695,...,10.272663,DE,0.009939,chr1p,123400000,0.991896,122400000,A2,1,DE
20,chr1,1000000,1050000,0.007767,True,DE-FA-DSG-DdeI-DpnII-P1P2.hg38.mapq_30.1000.mcool,0.007591,0.003267,0.004070,0.001037,...,10.278212,DE,0.009820,chr1p,123400000,0.991491,122350000,A2,1,DE
22,chr1,1100000,1150000,0.008009,True,DE-FA-DSG-DdeI-DpnII-P1P2.hg38.mapq_30.1000.mcool,0.007045,0.003385,0.003523,0.000547,...,10.323419,DE,0.009996,chr1p,123400000,0.990681,122250000,A2,1,DE
23,chr1,1150000,1200000,0.007971,True,DE-FA-DSG-DdeI-DpnII-P1P2.hg38.mapq_30.1000.mcool,0.007100,0.003616,0.003591,0.000261,...,10.331922,DE,0.007983,chr1p,123400000,0.990276,122200000,A2,1,DE


In [4]:
df.total_distance = (df.total_distance - df.total_distance.mean()) / df.total_distance.std()
#df.loc[df.total_distance > 2, "total_distance"] = 2
#df.loc[df.total_distance < 2, "total_distance"] = 0

In [5]:
dfs = lib.partition_by_sample(df, samples)
component = lib.init_scatters(
    samples=list(zip(samples, dfs)),
    # xy_options=("umap1_n500", "umap2_n500"),
    xy_options=("PCA1", "PCA2", "PCA3", "PCA4"),
    color_options=("IPG", "centel", "total_distance", "leiden", "weight", "PCA1", "PCA2", "chrom", "start"),
    color_kwargs={
        "weight": dict(
            norm=colors.Normalize(),
            map="bwr"
        ),
    }
)

# component.observe(lambda change: print(change.new), names=["coords"])

In [6]:
# Create HiGlass widget
conf = hg.Viewconf.parse_file("./viewconf.json")
bedfile_path = "/Users/thomas/joint-pca-hic/data/hepatic_differentiation_2024_02_07_09_07_PCA-32_50000bp_hg38_post_processed_ipgs.bed_ESC.bed.gz"
bins = bf.read_table(
    bedfile_path,
    schema="bed9",
    schema_is_strict=True,
)

# Create tileset
tileset = DynamicTileset(name="IPG clusters", chromsizes=bf.fetch_chromsizes('hg38')[:'chrY'])
tileset.update(bins)

# Create track
track = hg.server.add(tileset).track(height=30).opts(fillOpacity=1)
view = conf.views[0]
view.tracks.top[1].height= 100
view.tracks.top.append(track)
hg_viewer = conf.widget()


# Register callback on scatterplots
def update_tileset(coords):
    if len(coords) > 0:
        inds = bf.overlap(bins, coords, how="inner", return_index=True, return_input=False)
        tileset.update(bins.iloc[inds["index"]])
    else:
        tileset.update(bins)

def on_coords_change(change):
    update_tileset(change.new)
    hg_viewer.reload(dict(trackId=track.uid, viewId=view.uid))

component.observe(on_coords_change, names=["coords"])


ipywidgets.VBox([component, hg_viewer])

In [16]:
from higlass._scale import Scale

scale = Scale(bf.fetch_chromsizes("hg38"))
lo = int(scale(("chr3", 10_000_000)))
hi = int(scale(("chr3", 20_000_000)))
hg_viewer.zoom_to(view.uid, lo, hi)